In [84]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [4]:
#Initialiser le webdriver pour firefox
driver = webdriver.Firefox()

driver.get("https://www.carrefour.fr/promotions")



In [6]:
try:
    reject_all = driver.find_element(By.XPATH, "//*[@id='onetrust-reject-all-handler']")
    reject_all.click()
except Exception as e:
    print(f"Erreur : {e}")



In [8]:
import time
import csv
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# URL de base sans le numéro de page
base_url = "https://www.carrefour.fr/promotions?noRedirect=1&page="

# Listes pour stocker les informations
product_names = []
product_prices = []
quantities = []
ratings = []
reviews = []
promotions = []

page_num = 1  # Démarrer à la première page

# Boucle tant qu'il y a des pages à scraper
while True:
    print(f"Scraping page {page_num}")
    driver.get(base_url + str(page_num))  # Charger la page avec le numéro

    # Attendre un peu pour le chargement de la page
    time.sleep(3)
    
    # Récupérer les produits sur la page actuelle
    products = driver.find_elements(By.XPATH, "//article[contains(@class, 'product-card')]")
    
    if not products:
        print(f"Aucun produit trouvé sur la page {page_num}. Fin de la pagination.")
        break
    
    print(f"Nombre de produits trouvés sur la page {page_num}: {len(products)}")
    
    # Scraper les informations des produits sur cette page
    for p in products:
        # Localiser l'élément contenant le nom du produit
        try:
            name_element = p.find_element(By.XPATH, ".//h3[contains(@class, 'product-card-title__text')]")
            product_names.append(name_element.text)
        except NoSuchElementException:
            product_names.append("Nom non disponible")
        
        # Localiser l'élément contenant le prix du produit
        try:
            price_element = p.find_element(By.XPATH, ".//p[contains(@class, 'product-price__content')]")
            product_prices.append(price_element.text)
        except NoSuchElementException:
            product_prices.append("Prix non disponible")
        
        # Localiser l'élément contenant la quantité ou le format
        try:
            quantity_element = p.find_element(By.XPATH, ".//div[contains(@class, 'ds-format')]")
            quantities.append(quantity_element.text)
        except NoSuchElementException:
            quantities.append("Quantité non disponible")
        
        # Localiser l'élément contenant la note (étoiles)
        try:
            stars = p.find_element(By.XPATH, ".//span[@class='c-rating__stars']")
            full_stars = stars.find_elements(By.CLASS_NAME, "c-icon-star-fill")
            half_stars = stars.find_elements(By.CLASS_NAME, "c-icon-star-semi-fill")
            rating = len(full_stars) + 0.5 * len(half_stars)
            ratings.append(f"{rating} étoiles")
        except NoSuchElementException:
            ratings.append("Note non disponible")
        
        # Localiser l'élément contenant le nombre d'avis
        try:
            review_element = p.find_element(By.XPATH, ".//p[contains(@class, 'c-text--style-caption')]")
            reviews.append(review_element.text)
        except NoSuchElementException:
            reviews.append("Avis non disponible")
    
        # Localiser l'élément contenant la promotion ou la réduction
        try:
            promotion_element = p.find_element(By.XPATH, ".//span[contains(@class, 'promotion-label-refonte__text')]")
            promotions.append(promotion_element.text)
        except NoSuchElementException:
            promotions.append("Pas de promotion")

    # Passer à la page suivante
    page_num += 1

# Sauvegarder les informations extraites dans un fichier CSV
with open('carrefour_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Nom', 'Prix', 'Quantité', 'Note', 'Nombre d\'avis', 'Réduction'])
    for i in range(len(product_names)):
        writer.writerow([product_names[i], product_prices[i], quantities[i], ratings[i], reviews[i], promotions[i]])

print(f"{len(product_names)} produits ont été extraits et sauvegardés dans 'carrefour_products.csv'.")

Scraping page 1
Nombre de produits trouvés sur la page 1: 35
Scraping page 2
Nombre de produits trouvés sur la page 2: 35
Scraping page 3
Nombre de produits trouvés sur la page 3: 35
Scraping page 4
Nombre de produits trouvés sur la page 4: 35
Scraping page 5
Nombre de produits trouvés sur la page 5: 35
Scraping page 6
Nombre de produits trouvés sur la page 6: 35
Scraping page 7
Nombre de produits trouvés sur la page 7: 35
Scraping page 8
Nombre de produits trouvés sur la page 8: 35
Scraping page 9
Nombre de produits trouvés sur la page 9: 35
Scraping page 10
Nombre de produits trouvés sur la page 10: 35
Scraping page 11
Nombre de produits trouvés sur la page 11: 35
Scraping page 12
Nombre de produits trouvés sur la page 12: 35
Scraping page 13
Nombre de produits trouvés sur la page 13: 35
Scraping page 14
Nombre de produits trouvés sur la page 14: 35
Scraping page 15
Nombre de produits trouvés sur la page 15: 35
Scraping page 16
Nombre de produits trouvés sur la page 16: 35
Scraping p

In [10]:
import pandas as pd 

# Charger le fichier CSV
file_path = 'carrefour_products.csv'
data = pd.read_csv(file_path)

# Garde uniquement les colonnes nécessaires : Nom, Prix, Quantité, Réduction
cleaned_data = data[['Nom', 'Prix', 'Quantité', 'Réduction']].copy()

# Suppression des espaces superflus dans les noms des produits
cleaned_data.loc[:, 'Nom'] = cleaned_data['Nom'].str.strip()

# Suppression des espaces dans la colonne Quantité et normalisation
cleaned_data.loc[:, 'Quantité'] = cleaned_data['Quantité'].str.strip()

# Renommer la colonne 'Quantité' en 'Unité'
cleaned_data.rename(columns={'Quantité': 'Unité'}, inplace=True)

# Nettoyage de la colonne Réduction (par exemple, uniformiser les valeurs dans la colonne Réduction)
cleaned_data.loc[:, 'Réduction'] = cleaned_data['Réduction'].str.lower().str.strip()

# Affichage des premières lignes pour vérifier
print(cleaned_data.head())

# Exporter les données nettoyées si nécessaire
cleaned_data.to_csv('carrefour_product_final.csv', index=False)


                                                 Nom  Prix  \
0                Œufs de Plein Air Bio CARREFOUR BIO     5   
1  Papier Toilette Confort Extrait Lotus Aqua Tub...     8   
2     Barre chocolatée Chocolat au Lait KINDER BUENO     4   
3      Essuie-tout Classic Blanc CARREFOUR ESSENTIAL     2   
4  Café en grains Classique 100% arabica GIGA FOR...    21   

                     Unité         Réduction  
0                  15 œufs  2 achetés = 9,9€  
1  Quantité non disponible   25% d'économies  
2  Quantité non disponible    le 2ème à -60%  
3  Quantité non disponible    le 2ème à -30%  
4                      2kg   vu en catalogue  
